In [10]:
import pandas as pd
from scipy.stats import pearsonr

In [7]:
mv_history = pd.read_csv('./data/pipeline_history.csv')

In [14]:
sum(mv_history.Units * mv_history.Built) / mv_history.Units.sum()

0.6719286871961102